# Monad transformers

## Outline

* Incentive for the Monad transformers

* The MonadIO type class

* Monad Transformers
  * StateT 

  * WriterT

  * ReaderT

  * MaybeT

  * EitherT

In this lesson, we will learn about Monad transformers and how you can use them. First we show the incentive for Monad transformers where we solve a problem with the help of the MonadIO type class and explain it. Then we introduce Monad transformers and show how these kind of problems can be solved with them. 

## Incentive for Monad transformers

In the homework of the State monad lesson we learned how you can implement the Tic-Tac-Toe game with using the State monad. 

A curious student might ask how to implement this game with the State monads by geting the input from the user instead of the random funtion.

Or in other words, is it possible to perform IO actions within a user defined State monad? The answer is yes.

With the knowledge that we have so far we can create a custom State monad that we will call `StateIO`.

The idea is that the result of the `runState` function will have to be wrapped in IO.

Below is the code that implements our costum `StateIO` state monad that returns the result in IO context.

In [ ]:
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE FlexibleInstances #-} 

import Control.Monad.Trans
import Control.Monad (ap, liftM)
import Control.Monad.State.Class (MonadState(get, put))

newtype StateIO s a = StateIO {runStateIO :: s -> IO (a, s)}

instance Functor (StateIO s) where
    fmap = liftM

instance Applicative (StateIO s) where
    pure = return
    (<*>) = ap

instance Monad (StateIO s) where
    return a = StateIO $ \s -> return (a, s)
    n >>= k = StateIO $ \s -> do (a, s') <- runStateIO n s
                                 runStateIO (k a) s'

instance MonadState s (StateIO s) where
    get = StateIO $ \st -> return (st,st)
    put st = StateIO $ const $ return ((),st)

We see that the `StateIO` type is defined similar as the `State` type just that it returns `IO (a,s)` instead of `(a,s)`.

But if we try now and create a state IO monad and try to perform some IO action in it, the code will not work.

In [ ]:
data Choice = Empty | X | O deriving (Eq, Show)
type Board = [Choice]

getUserMove :: StateIO Board ()
getUserMove = do
    board <- get
    putStrLn "Make your move X player (1-9):" -- causes compilation error
    choice <- getLine -- also causes compilation error
    let validMove = length choice == 1 && all (`elem` "123456789") choice
    if validMove
    then do
        let newBoar = [mv |ind <- [1..9], let mv = if ind == read choice then X else Empty]
        put newBoar  
    else do
        putStrLn "Not a valid choice. Make new one." -- also causes compilation error
        getUserMove
    return ()

main :: IO ()
main = do
    let board = [Empty | boardInd <- [1..9]]
    (_, newBoard) <- runStateIO getUserMove board
    putStrLn $ "New Board is: " ++ show newBoard

main

The reason for this is how the bind operator for the `StateIO` is defined. 
```haskell
(>>=) :: StateIO s a -> (a -> StateIO s b) -> StateIO s b
```

It can not take in a non `StateIO` variable variable or a function that wouldn't return a `StateIO` variable. 

The `print` function returns only `IO ()` and not `StateIO () a`, so the bind operator can not be applied to it.

The solution to this is defining a function that takes an IO action and lifts it to a `StateIO` monad.

In [ ]:
liftIOtoStateIO :: IO a -> StateIO s a
liftIOtoStateIO io = StateIO $ \st -> do x <- io
                                         return (x, st)

Now we can write again our code and it will compile correctly.

In [ ]:
data Choice = Empty | X | O deriving (Eq, Show)
type Board = [Choice]

getUserMove :: StateIO Board ()
getUserMove = do
    board <- get
    liftIOtoStateIO $ putStrLn "Make your move X player (1-9):"
    choice <- liftIOtoStateIO getLine 
    let validMove = length choice == 1 && all (`elem` "123456789") choice
    if validMove
    then do
        let newBoar = [mv |ind <- [1..9], let mv = if ind == read choice then X else Empty]
        put newBoar  
    else do
        liftIOtoStateIO $ putStrLn "Not a valid choice. Make new one."
        getUserMove
    return ()

main :: IO ()
main = do
    let board = [Empty | boardInd <- [1..9]]
    (_, newBoard) <- runStateIO getUserMove board
    putStrLn $ "New Board is: " ++ show newBoard

main

We created now a State monad that works in IO context and found a way how to perform IO actions inside of such a monad. 

Of course you could do the same and create a State monad that returns the result in a Maybe context or as a list of tuples.

Such transformations make code more practical. For this reason Haskell provides build in solutions for them.

## The MonadIO type class

Because lifting a type wraped inside IO to another monad is very usefull, Haskell handles this with the `MonadIO` type class.

It defines the `liftIO` function that generalizes the type signature of our `liftIOtoStateIO` function.
```haskell
class Monad m => MonadIO m where
    liftIO :: IO a -> m a 
```

The `MonadIO` type class is defined in the **Control.Monad.IO.Class** module which is part of the **base** package.

So insted of defining a custom function for lifting we can import this module and create an instance of `MonadIO` for the `StateIO s` type.

In [ ]:
import Control.Monad.IO.Class

instance MonadIO (StateIO s) where
    liftIO io = StateIO $ \st -> do x <- io
                                    return (x, st)

getUserMove :: StateIO Board ()
getUserMove = do
    board <- get
    liftIO $ putStrLn "Make your move X player (1-9):"
    choice <- liftIO getLine 
    let validMove = length choice == 1 && all (`elem` "123456789") choice
    if validMove
    then do
        let newBoar = [mv |ind <- [1..9], let mv = if ind == read choice then X else Empty]
        put newBoar  
    else do
        liftIO $ putStrLn "Not a valid choice. Make new one."
        getUserMove
    return ()

## Monad transformers

We ask ourselfs now how to generalize wrapping the result of the State monad function into a monadic context.

When trying to wrap an existing monad into another monad we call the function that does this a **monad transformer**.

This can be done not just for the State and the IO Monad, but all possible monad combinations can be handled with Monad transformers.

From [Haskell wikibook](https://en.wikibooks.org/wiki/Haskell/Monad_transformers): Monad transformers are special types that allow us to roll two monads into a single one that shares the behavior of both. 

There are many transformers already defined in various modules. We will cover here the following transformers:
- State monad transformer `StateT`

- Writer monad transformer `WriterT`

- Reader monad transformer `ReaderT`

- Maybe monad transformer `MaybeT`

- Either monad transformer `EitherT`

## StateT monad transformer

For the State monad we can use the `StateT` monad transformer, which is provided in the **Control.Monad.Trans.State.Lazy** module

It is defined similar as the State monad type. The module comes with the **transformers** package.

```haskell
newtype StateT s m a = StateT { runStateT :: s -> m (a,s) }

instance Monad m => Monad (StateT s m) where
    return a = StateT $ \ s -> return (a, s)
    m >>= k  = StateT $ \ s -> do
        ~(a, s') <- runStateT m s
        runStateT (k a) s'
```

Here the `~` operator in the above expression means ... TODO 

When defining an Functor and Applicative instance for this type we also need to add the Monad type constraint.
```haskell
instance Monad m => Functor (StateT s m) where
    fmap = liftM 

instance Monad m => Applicative (StateT s m) where
    pure = return
    (<*>) = ap
```

### Examples

Now we can re-write the initial code example with the `StateIO` monad by using the `StateT` monad instead.

In [ ]:
import Control.Monad.Trans.State.Lazy ( StateT(runStateT), get, put )
import Control.Monad.IO.Class ( MonadIO(liftIO) )

data Choice = Empty | X | O deriving (Eq, Show)
type Board = [Choice]

getUserMove :: StateT Board IO ()
getUserMove = do
    board <- get
    liftIO $ putStrLn "Make your move X player (1-9):"
    choice <- liftIO getLine 
    let validMove = length choice == 1 && all (`elem` "123456789") choice
    if validMove
    then do
        let newBoar = [mv |ind <- [1..9], let mv = if ind == read choice then X else Empty]
        put newBoar  
    else do
        liftIO $ putStrLn "Not a valid choice. Make new one."
        getUserMove
    return ()

main :: IO ()
main = do
    let board = [Empty | boardInd <- [1..9]]
    (_, newBoard) <- runStateT getUserMove board
    putStrLn $ "New Board is: " ++ show newBoard

We see there is not need to define any custom `State` monad and make instances for it. We simply use the `StateT` monad with the `IO` context.

Another benifit is we do not have to declare an `MonadIO` instance for `StateT` because it allready has it declared in the module that defines it.

Also we can now also understand the official Haskell definition of the `State` monad:
```haskell
type State s = StateT s Identity
```
We see the `State` monad is defined in terms of the `StateT` monad transformer.

The monad used in the `StateT` transformer is the `Identity` monad, which is only a wrapper for a variable of type `a`:
```haskell
newtype Identity a = Identity { runIdentity :: a }
```

But as seen in the example above we can use the `IO` monad or any other instead of the `Identitiy` monad which gives us the basic `State` monad.

## WriterT monad transformer

## Recap

In this lesson we've discussed:

- the motivation for introducing Monad transformers

- The MonadIO type class

- Monad Transformers